In [1]:
import transformers # HuggingFace!
from transformers import AutoTokenizer # a librabry that has tokenizers for all of the different models
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
#import pdb

/Users/wjones/opt/anaconda3/envs/py4sci/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-25 11:07:03.658538: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to /Users/wjones/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
#!pip install pytorch_pretrained_bert

from pytorch_pretrained_bert import BertTokenizer, BertModel
from transformers import AutoModelForSequenceClassification

BERT_CLASS = AutoModelForSequenceClassification

# Make sure all the files are in same folder, i.e vocab , config and bin file
PRE_TRAINED_MODEL_NAME_OR_PATH = '/path/to/the/files/containing/models/files'

model = BERT_CLASS.from_pretrained('ert.pth', num_labels = 2)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [4]:
#Read a comma-separated values (csv) file into DataFrame
path = "clean_IMDBdataset.csv"
df = pd.read_csv(path, sep=",")
#This changes all of the Y labels to be 0s or 1s
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(df, df, test_size=0.2, random_state=30)

X_test,X_valid,Y_test, Y_valid = train_test_split(X_test, Y_test, test_size=0.5, random_state=30)

X_tr_dataset = Dataset.from_pandas(X_train)
X_test_dataset = Dataset.from_pandas(X_test)
X_v_dataset = Dataset.from_pandas(X_valid)

def tokenize_function(examples):
    return tokenizer(examples["Phrase"], padding="max_length", truncation=True)


# Tokenizes the data
tokenizedX_train = X_tr_dataset.map(tokenize_function, batched=True)
tokenizedX_test = X_test_dataset.map(tokenize_function, batched=True)
tokenizedX_valid = X_v_dataset.map(tokenize_function, batched=True)
# Remove the column with the actual reviews (as opposed to the review ids) because the model does not accept raw text as an input!
tokenizedX_trainE = tokenizedX_train.remove_columns(["Phrase"])
tokenizedX_testE = tokenizedX_test.remove_columns(["Phrase"])
tokenizedX_validE = tokenizedX_valid.remove_columns(["Phrase"])

# Removes another unnecesary column
tokenizedX_trainE = tokenizedX_trainE.remove_columns(["__index_level_0__"])
tokenizedX_testE = tokenizedX_testE.remove_columns(["__index_level_0__"])
tokenizedX_validE = tokenizedX_validE.remove_columns(["__index_level_0__"])


# Makes sure the columns are titled in the way that the model expects
tokenizedX_trainf = tokenizedX_trainE.rename_column("Sentiment", "labels")
tokenizedX_testf = tokenizedX_testE.rename_column("Sentiment", "labels")
tokenizedX_validf = tokenizedX_validE.rename_column("Sentiment", "labels")



# Sets the format of the dataset to return PyTorch tensors instead of lists
tokenizedX_trainf.set_format("torch")
tokenizedX_testf.set_format("torch")
tokenizedX_validf.set_format("torch")
# You might need to create a smaller subset of the dataset if it's going really slow

train_dataloader = DataLoader(tokenizedX_trainf, shuffle=True, batch_size=8)
# I don't want the eval DataLoader to be my validation, right?
valid_dataloader = DataLoader(tokenizedX_validf, batch_size=8)


In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fa

In [14]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in valid_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()